In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os
sys.path.append(os.pardir)

In [3]:
import gc

In [4]:
import pdb

In [5]:
import numpy as np
import torch
import umap

In [49]:
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size':16})

In [6]:
from data.dataset import bird_dataset

In [7]:
from utils.utils import load_netG, overlap_encode, overlap_decode, segment_image

# load a bird, get its songs from all days

In [8]:
path2bird = '/home/gagan/data/zudeep/bird_hdfs/r15y2'
dataset = bird_dataset(path2bird)

...total number of folders = 43 ...


In [9]:
X_real = []

for n in range(dataset.nfolders):
    
    seqs = dataset.get(day = n, nsamps=-1)
    for s in seqs:
        segs = segment_image(s, width=16)
        segs = [np.ndarray.flatten(s) for s in segs]
        X_real += segs
        
        

... total available files = 5 ...
... total available files = 220 ...
... total available files = 316 ...
... total available files = 340 ...
... total available files = 644 ...
... total available files = 877 ...
... total available files = 953 ...
... total available files = 1315 ...
... total available files = 1093 ...
... total available files = 517 ...
... total available files = 1101 ...
... total available files = 1338 ...
... total available files = 1544 ...
... total available files = 1463 ...
... total available files = 1203 ...
... total available files = 2005 ...
... total available files = 1421 ...
... total available files = 1648 ...
... total available files = 1355 ...
... total available files = 2068 ...
... total available files = 1773 ...
... total available files = 2351 ...
... total available files = 2301 ...
... total available files = 2277 ...
... total available files = 1740 ...
... total available files = 1707 ...
... total available files = 1778 ...
... total a

In [10]:
len(X_real)

1489096

In [11]:
X_real[0].shape

(2064,)

In [12]:
X_real = np.stack(X_real, axis = 0)

In [13]:
X_real.shape

(1489096, 2064)

In [14]:
gc.collect()

17

In [16]:
# save 
np.save('/home/gagan/data/zudeep/processed_files/r15y2_all_spect_chunks.npy', X_real)

In [17]:
# subsample

ids = np.random.choice(len(X_real),size=200000,replace=False)

X_real = X_real[ids]

In [21]:
X_real.shape

(200000, 2064)

In [51]:
np.save('/home/gagan/data/zudeep/processed_files/r15y2_subsampled_200000_spect_chunks.npy', X_real)

In [83]:
X_real = np.load('/home/gagan/data/zudeep/processed_files/r15y2_subsampled_200000_spect_chunks.npy')

# create network, generate samples

In [20]:
netG = load_netG('/home/gagan/data/zudeep/models/nz_12_residual/2021-05-05_10-41-37.547480/netG_epoch_5.pth', nz=12, ngf=128, cuda=True, resnet=True)

In [114]:
Z = 2.*np.random.randn(200000, 12)

#Z = -10 + 20*np.random.rand(200000, 12)
Z.shape

(200000, 12)

In [115]:
Xhat,_ = overlap_decode(Z, netG)

In [116]:
Xhat.shape

(129, 3200000)

In [117]:
segs = segment_image(Xhat, width=16)
Xhat = [np.ndarray.flatten(s) for s in segs]
del segs
gc.collect()

34

In [118]:
Xhat = np.stack(Xhat,axis=0)

In [119]:
Xhat.shape

(200000, 2064)

# project on umap

In [53]:
del X_real
gc.collect()

15742

In [45]:
from umap import UMAP

In [124]:
umaplearner = UMAP(n_neighbors=15, n_components=2, min_dist = 0.01)

In [ ]:
xfake2 = umaplearner.fit_transform(Xhat)

In [ ]:
xreal2 = umaplearner.fit_transform(X_real)

In [ ]:
xreal2 = umaplearner.transform(X_real)

In [98]:
xfake2 = umaplearner.transform(Xhat)

In [ ]:
plt.figure(figsize=(15,15))
plt.scatter(xreal2[:,0], xreal2[:,1], s = 3)
plt.scatter(xfake2[:,0], xfake2[:,1], s = 3, c = 'w', marker='o', edgecolors='r', alpha=0.3)
plt.legend(['real', 'sampled'])
#plt.savefig('/home/gagan/Dropbox/Work/MDGAN_paper/Figures_for_paper/Final_figures/Figure1/umap_nz12_netGsamples_Variance2_vs_realdata_r15y2_200000_subsampled.png', format='png', dpi=200)
#plt.close()

In [107]:
import seaborn as sns

In [108]:
import pandas as pd

In [109]:
df_real = pd.DataFrame({'x1': xreal2[:,0], 'x2': xreal2[:,1]})
df_fake = pd.DataFrame({'x1': xfake2[:,0], 'x2': xfake2[:,1]})


In [ ]:
plt.figure(figsize=(15,15))
sns.displot(df_real, x="x1", y="x2", kind="kde")
sns.displot(df_fake, x="x1", y="x2", kind="kde")